In [ ]:
import pandas as pd

# === 1. Load CSV ===
filename = "AGGREGATED_GENERATION_PER_TYPE_GENERATION_15min_2023.csv"
outputfile = "gen_hourly_MW_2023.csv"
df = pd.read_csv(filename, low_memory=False)

# === 2. Parse the MTU column to extract start/end times ===
mtu = df["MTU (CET/CEST)"].str.extract(
    r'(?P<start>\d{2}/\d{2}/\d{4} \d{2}:\d{2}:\d{2})\s*(?:\((?:CET|CEST)\))?\s*-\s*'
    r'(?P<end>\d{2}/\d{2}/\d{4} \d{2}:\d{2}:\d{2})\s*(?:\((?:CET|CEST)\))?'
)

df["Start"] = pd.to_datetime(mtu["start"], dayfirst=True, errors="coerce")
df["End"] = pd.to_datetime(mtu["end"], dayfirst=True, errors="coerce")

# === 3. Convert Generation column to numeric ===
gcol = "Generation (MW)"
df[gcol] = pd.to_numeric(df[gcol].astype(str).str.replace(",", "."), errors="coerce")

# === 4. Clean and prepare data ===
df = df.dropna(subset=["Start", gcol]).set_index("Start").sort_index()

# === 5. Define grouping columns (if present) ===
group_cols = [c for c in ["Area", "Production Type"] if c in df.columns]

# === 6. Resample to hourly average power (MW) ===
hourly_avg = (
    df.groupby(group_cols + [pd.Grouper(freq="1h")])[gcol]
      .mean()
      .reset_index()
      .rename(columns={"Start": "datetime", gcol: "Generation (MW)"})
)

# === 7. Reorder columns: put datetime first ===
cols = ["datetime"] + [c for c in hourly_avg.columns if c != "datetime"]
hourly_avg = hourly_avg[cols]

# === 8. Save result (optional) ===
hourly_avg.to_csv(outputfile, index=False)

print(hourly_avg.head())